## Multi Object Tracking sample
This notebook is an example of how to perform object detection with multi-object tracking (MOT) from a video file to count vehicle traffic. The annotated video is saved into new file with `_mot_annotated` suffix.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

You also need to specify your cloud API access token in [env.ini](../../env.ini) file, located in the same repository as this notebook.

In [ ]:
# make sure degirum-tools package and other dependencies are installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want to run your inferences, model zoo url, model name and image source

In [ ]:
# hw_location: where you want to run inference
#     @cloud to use DeGirum cloud
#     @local to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# input_filename: path to the video file to annotate
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
input_filename = "../../images/Traffic.mp4"

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools
from pathlib import Path
import IPython.display

# configure for Google Colab
degirum_tools.configure_colab()

# connect to AI inference engine getting token from env.ini file
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load object detection model
model = zoo.load_model(
    model_name, overlay_show_probabilities=True, overlay_line_width=1
)

# video input and output
orig_path = Path(input_filename)
ann_path = "../../workarea/" + orig_path.stem + "_mot_annotated" + orig_path.suffix



# create object tracker
tracker = degirum_tools.ObjectTracker(
    class_list=["car"],
    track_thresh=0.35,
    track_buffer=100,
    match_thresh=0.9999,
    trail_depth=20,
    anchor_point=degirum_tools.AnchorPoint.BOTTOM_CENTER,
)

# create line counter
line_counter = degirum_tools.LineCounter([(120, 430, 870, 430), (860, 80, 860, 210)])

# annotate video applying object tracker and line counter
degirum_tools.annotate_video(
    model, orig_path, ann_path, analyzers=[tracker, line_counter]
)

In [ ]:
# display result
IPython.display.Video(filename=str(ann_path), embed=degirum_tools._in_colab())

In [ ]:
# display original video
IPython.display.Video(filename=str(orig_path), embed=degirum_tools._in_colab())